In [80]:
import pandas as pd
from sklearn import linear_model

In [81]:
train = pd.read_csv('./data/train.csv')
train

PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
8688     9276_01     Europa     False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0             0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1           109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2            43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3             0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4           303.0       70.0         151.0   565.0     2.0  Willy Santantines   
...           ...        ...           ...     ...     ...                ...   
8688          0.0     6819.0           0.0  1643.0    74.0  Gravior Noxnuther   
8689          0.0        0.0           0.0     0.0     0.0    Kurta Mondalley   
8690          0.0        0.0        1872.0     1.0     0.0       Fayey Connon   
8691          0.0     1049.0           0.0   353.0  3235.0   Celeon Hontichre   
8692        126.0     4688.0           0.0     0.0    12.0   Propsh Hontichre   

      Transported  
0           False  
1            True  
2           False  
3           False  
4            True  
...           ...  
8688        False  
8689        False  
8690         True  
8691        False  
8692         True  

[8693 rows x 14 columns]

In [82]:
train["Cabin"].unique()

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      shape=(6561,), dtype=object)

In [83]:
def pipeline(df):
    df = df.drop(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'], axis=1)
    
    df[['Deck', 'Number', 'Side']] = df['Cabin'].str.split('/', expand=True)
    df = df.drop("Cabin", axis=1)

    deck_dummies = pd.get_dummies(df['Deck'], prefix='Deck')
    df = pd.concat([df, deck_dummies], axis=1)
    df = df.drop("Deck", axis=1)

    df['Side_bool'] = df['Side'].map({'S': True, 'P': False})
    df = df.drop("Side", axis=1)

    df = df.dropna()

    return df


train = pipeline(train)
y = train['Transported']
x = train.drop(["Transported", "PassengerId"], axis=1)

In [84]:
reg = linear_model.LinearRegression()
reg.fit(x, y)

LinearRegression()

In [85]:
test = pd.read_csv('./data/test.csv')
test = pipeline(test)

predictions = reg.predict(test.drop("PassengerId", axis=1))

prediction_bool = predictions >= 0.5

In [86]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Transported": prediction_bool
})


submission.to_csv("submission.csv", index=False)